In [ ]:
# How Config Files (dev/local YAMLs) Point Code to Different Sources
# ------------------------------------------------------------------

import yaml
import os
import boto3

# 1. Example config.yaml content (simulated here)
yaml_config = """
local:
  s3_input: data/input/
  db_conn: sqlite:///:memory:
  kafka_topic: local-orders

dev:
  s3_input: s3://my-dev-bucket/raw/
  db_conn: dev-cluster.region.redshift.amazonaws.com
  kafka_topic: dev-orders

prod:
  s3_input: s3://my-prod-bucket/raw/
  db_conn: prod-cluster.region.redshift.amazonaws.com
  kafka_topic: prod-orders
"""

# 2. Load config (in practice, load from file)
config = yaml.safe_load(yaml_config)

# 3. Detect current environment
env = os.getenv("ENV", "local")
settings = config.get(env, {})

print(f"🚀 Running in {env.upper()} environment")
print("Configuration loaded:")
print(settings)

# 4. Example: Point code to correct S3 source
if env == "local":
    print("\nReading data from:", settings["s3_input"])
else:
    print("\nFetching data from S3 source:", settings["s3_input"])

# 5. Example: Database connection (simulation)
print("\nConnecting to DB:", settings["db_conn"])
print("Using Kafka Topic:", settings["kafka_topic"])

# 6. Optional: Fetch DB password securely via AWS SSM
try:
    ssm = boto3.client("ssm")
    password = ssm.get_parameter(Name=f"/{env}/db/password", WithDecryption=True)["Parameter"]["Value"]
    print(f"Fetched DB password for {env}: {password[:4]}****")
except Exception as e:
    print("\n⚠️ Skipping secret fetch (simulation mode):", e)

# 7. Summary
"""
- Each environment block (local/dev/prod) defines separate sources.
- YAML config points code to correct input/output (S3, DB, Kafka).
- Secrets are fetched securely using AWS SSM.
- Simple, readable, and CI/CD friendly configuration pattern.
"""
